# Data Pre-processing

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten, MaxPool1D, Dropout, Reshape
from sklearn.metrics import classification_report

Using TensorFlow backend.


In [2]:
weather_raw_df = pd.read_csv('the_ultimate_complete_data_delet.csv',',',header=0,dtype='float64')
weather_raw_df.drop(['Belfort Rain Gauge','Logger Battery Voltage','Year','Julian Day','Time'],axis=1,inplace=True)

weather_labels = weather_raw_df['Precipitation (Tipping Bucket) 3H'].copy()

weather_raw_df.drop(['Precipitation (Tipping Bucket) 3H'],axis=1,inplace=True)

weather_raw_df.head(10)

,Net Radiometer,Incoming Shortwave Radiation,Reflected Shortwave Radiation,Relative Humidity,Ambient Air Temperature,Barometric Pressure,Wind Direction,Wind Speed,Wind Speed at Rain Gauge,Precipitation (Tipping Bucket)
0,-3.076,-2.500,0.000,61.36,2.046,101.4,107.9,2.639,2.217,9.0
1,-3.080,-2.500,-0.629,61.69,1.972,101.4,109.9,2.082,1.647,9.0
2,-3.085,-2.500,0.000,62.97,1.822,101.4,97.5,1.986,1.491,9.0
3,-3.089,-2.500,0.000,62.50,1.699,101.4,91.7,2.513,1.975,9.0
4,-3.094,-2.500,0.000,63.37,1.737,101.3,92.5,3.000,2.501,9.0
5,-3.097,-2.500,0.000,65.99,1.371,101.3,95.5,2.406,1.848,9.0
6,-3.100,-2.500,0.000,67.20,1.436,101.3,94.9,2.722,2.252,9.0
7,-3.102,-1.875,0.000,65.39,1.531,101.3,92.9,2.956,2.437,9.0
8,-3.104,-2.500,0.000,65.73,1.559,101.3,97.7,3.148,2.680,9.0
9,-3.105,-2.500,0.000,65.12,1.549,101.3,99.9,3.613,3.109,9.0


In [3]:
mask = weather_raw_df['Precipitation (Tipping Bucket)'] > 0
mask2 = weather_raw_df['Precipitation (Tipping Bucket)'] <= 0
weather_raw_df.loc[mask,'Precipitation (Tipping Bucket)'] = 1
weather_raw_df.loc[mask2,'Precipitation (Tipping Bucket)'] = 0

In [4]:
i = 0
for column in weather_raw_df:
    if i < 9:
        weather_raw_df.loc[:,column] += np.absolute(weather_raw_df[column].min())
        weather_raw_df.loc[:,column] /= weather_raw_df[column].max()
        weather_raw_df.loc[:,column] -= np.mean(weather_raw_df[column])
        #weather_raw_df.loc[:,column] /= np.std(weather_raw_df[column])
    i += 1

In [5]:
mask = weather_labels > 0
mask2 = weather_labels <= 0
weather_labels.loc[mask] = 1
weather_labels.loc[mask2] = 0

In [6]:
#one-hot-encoding
weather_labels_ohe = pd.get_dummies(weather_labels)

# K-Fold Cross Validation

In [7]:
from sklearn.model_selection import train_test_split,StratifiedKFold,cross_validate,cross_val_score

In [8]:
kfold = StratifiedKFold(n_splits=5,shuffle=True,random_state=42)

In [ ]:
x_train = weather_raw_df
y_train = weather_labels
y_train_ohe = weather_labels_ohe

# ANN Model

In [ ]:
from keras.wrappers.scikit_learn import KerasClassifier

In [ ]:
def model_const():
    model = Sequential([
             Reshape((1,10), input_shape=(10,)),
             Dense(2048, activation='relu',input_shape=(1,10)),
             MaxPool1D(1),
             Flatten(),
             Dense(2048, activation='relu'),
             Dropout(0.25),
             Dense(2048, activation='relu'),
             Dropout(0.25),
             Dense(2048, activation='relu'),
             Dense(2, activation='softmax')
        ])
    model.compile(optimizer='sgd',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    return model

ANN = KerasClassifier(build_fn=model_const, epochs=10, batch_size=10, verbose=0)

In [ ]:
ann_score = cross_validate(ANN, x_train, y_train, scoring=['accuracy','f1'],cv=kfold)

In [ ]:
for key in ann_score:
    print(key + ": " + str(np.average(ann_score[key])))

# SVM Model

In [ ]:
from sklearn import svm

In [ ]:
# Polynomial degree 4, C=1
SVM = svm.SVC(kernel='poly',degree=4,C=1,gamma='scale', verbose=True)

In [ ]:
svm_score = cross_validate(SVM, x_train, y_train, scoring=['accuracy','f1'],cv=kfold)

[LibSVM]

In [ ]:
for key in svm_score:
    print(key + ": " + str(np.average(svm_score[key])))

# Random Forest Model

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
#auto (n) features
RF = RandomForestClassifier(n_estimators=500,max_features=None,random_state=42,verbose=1)

In [ ]:
rf_score = cross_validate(RF, x_train, y_train,scoring=['accuracy','f1'],cv=kfold)

In [ ]:
for key in rf_score:
    print(key + ": " + str(np.average(rf_score[key])))